<a href="https://colab.research.google.com/github/rdemarqui/Data-Science-Basico/blob/master/Whisper_Fine_tunning_Portuguese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper Fine-tunning

In [ ]:
# To do
# Evaluate before training
# Download streaming mode dataset
# Merge more than one dataset
# Transform audio into Mel Spectogram
# Data Augmentation
# Normalize text
# Drop audio > 30s
# Drop large embedding
# Save checkpoints
# Evaluate after training (WER)
# Compare with leaderboard: https://huggingface.co/spaces/whisper-event/winners?dataset=mozilla-foundation%2Fcommon_voice_11_0
# Save final model
# Create a App(?)


In [ ]:
! pip install transformers

In [3]:
# Check Environment
device = !nvidia-smi -L
device = "cuda" if device[0][:3] == 'GPU' else "cpu"
device

'cpu'

## Datasets

In [ ]:
AUDIO_COLUMN_NAME = "audio"
TEXT_COLUMN_NAME = "sentence"

In [ ]:
# Whisper was trained with 16KHz samplig rate. We will resample the dataset to guarantee this.

In [ ]:
from datasets import Audio, DatasetDict, concatenate_datasets, load_dataset

def normalize_dataset(ds, audio_column_name=None, text_column_name=None):
    if audio_column_name is not None and audio_column_name != AUDIO_COLUMN_NAME:
        ds = ds.rename_column(audio_column_name, AUDIO_COLUMN_NAME)
    if text_column_name is not None and text_column_name != TEXT_COLUMN_NAME:
        ds = ds.rename_column(text_column_name, TEXT_COLUMN_NAME)
    # resample to the same sampling rate
    ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
    # normalise columns to ["audio", "sentence"]
    ds = ds.remove_columns(set(ds.features.keys()) - set([AUDIO_COLUMN_NAME, TEXT_COLUMN_NAME]))
    return ds

In [ ]:
raw_datasets = DatasetDict()

ds_train_mcv = load_dataset("mozilla-foundation/common_voice_11_0", "fr", split="train+validation", use_auth_token=True)
ds_train_mcv = normalize_dataset(ds_train_mcv)

ds_train_mls = load_dataset("facebook/multilingual_librispeech", "french", split="train+validation")
ds_train_mls = normalize_dataset(ds_train_mls, text_column_name="text")

raw_datasets["train"] = concatenate_datasets([ds_train_mcv, ds_train_mls])

# NB: shuffle concatenated dataset
raw_datasets["train"] = raw_datasets["train"].shuffle(seed=10)

raw_datasets["eval"] = load_dataset("mozilla-foundation/common_voice_11_0", "fr", split="test", use_auth_token=True)
raw_datasets["eval"] = normalize_dataset(raw_datasets["eval"])

raw_datasets

## Load Model

In [ ]:
# openai/whisper-small

In [ ]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

In [ ]:
# include forced token in the training
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
# to use gradient checkpointing
model.config.use_cache = False